# Class TemporalGraph

In [1]:
from functools import lru_cache
import os
import json
import networkx as nx
class TemporalGraph:
    def __init__(self, files):
        """
        Initialize TemporalGraph with a list of JSON files representing graphs at different timestamps.
        """
        self.files = files  # List of JSON file paths

    @lru_cache(maxsize=10)  # Cache the last 10 accessed timestamps
    def load_graph_at_timestamp(self, timestamp):
        """
        Load the graph for a specific timestamp from JSON and convert it to a NetworkX graph.
        """
        with open(self.files[timestamp], 'r') as f:
            data = json.load(f)
        return self._json_to_graph(data)

    def _json_to_graph(self, data):
        """
        Convert JSON data to a NetworkX graph.
        """
        graph = nx.DiGraph() if data["directed"] else nx.Graph()
        
        # Add nodes
        
        for node_type, nodes in data["node_values"].items():
            for node in nodes:
                node_id = node[-1]  # Assuming the node ID is the last element in the list
                node_attributes = dict(zip(data["node_types"][node_type], node))
                graph.add_node(node_id, **node_attributes)  # Add the node with its attributes
            
                
        # Add edges
        all_edge_types = data["relationship_types"]

        for i in data["relationship_values"] :
            
            if i[0] in all_edge_types :
                
                attributes = {}
                for j in range(len(i)-2) :
                    key = all_edge_types[i[0]][j]
                    attributes[key] = i[j]

                graph.add_edge(i[-2],i[-1],**attributes)
            else :
                graph.add_edge(i[0],i[1])
    
        
        return graph



# Loading and Wrapper

In [2]:
import time  # Ensure this is imported properly
import tracemalloc
import functools

def time_and_memory(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        # Start tracking memory and time
        tracemalloc.start()
        start_time = time.time()  # Ensure time module is used correctly
        
        try:
            # Call the actual function
            result = func(*args, **kwargs)
        finally:
            # Calculate memory and time usage
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            end_time = time.time()
            elapsed_time = end_time - start_time

            # Print results
            print(f"Time taken by '{func.__name__}': {elapsed_time:.2f} seconds")
            print(f"Memory used by '{func.__name__}': {current / 1024:.2f} KiB (Current), {peak / 1024:.2f} KiB (Peak)")

        return result
    return wrapper

In [3]:
import glob
import re

# Natural sorting function
def natural_sort(files):
    # Extract numeric parts from filenames for sorting
    return sorted(files, key=lambda x: int(re.search(r'timestamp_(\d+)', x).group(1)))

# Get files and sort
files = glob.glob("data/supply_chain_export_1000/timestamp_*.json")
files = natural_sort(files)

# Initialize TemporalGraph
temporal_graph = TemporalGraph(files)
temporal_graph.files

['data/supply_chain_export_1000\\timestamp_0.json',
 'data/supply_chain_export_1000\\timestamp_1.json',
 'data/supply_chain_export_1000\\timestamp_2.json',
 'data/supply_chain_export_1000\\timestamp_3.json',
 'data/supply_chain_export_1000\\timestamp_4.json',
 'data/supply_chain_export_1000\\timestamp_5.json',
 'data/supply_chain_export_1000\\timestamp_6.json',
 'data/supply_chain_export_1000\\timestamp_7.json',
 'data/supply_chain_export_1000\\timestamp_8.json',
 'data/supply_chain_export_1000\\timestamp_9.json',
 'data/supply_chain_export_1000\\timestamp_10.json',
 'data/supply_chain_export_1000\\timestamp_11.json']

# Queries

### Transportation cost between a warehouse and supplier.

In [4]:
@time_and_memory
def query_transportation_cost_for_supplier_and_warehouse(temporal_graph,timestamp, supplier_id, warehouse_id):
    G = temporal_graph.load_graph_at_timestamp(timestamp)
    if supplier_id in G.nodes and warehouse_id in G.nodes:
        if G.nodes[supplier_id].get("node_type") == "Supplier" and G.nodes[warehouse_id].get("node_type") == "Warehouse":
            if G.has_edge(supplier_id, warehouse_id):
                edge_data = G[supplier_id][warehouse_id]
                if edge_data.get("relationship_type") == "SupplierToWarehouse":
                    return edge_data.get("transportation_cost")
    return None


In [5]:
supplier_id = 'S_002'
warehouse_id = 'W_044'

transportation_cost = query_transportation_cost_for_supplier_and_warehouse(temporal_graph,0, supplier_id, warehouse_id)
print("Transportation cost:",transportation_cost)

Time taken by 'query_transportation_cost_for_supplier_and_warehouse': 0.34 seconds
Memory used by 'query_transportation_cost_for_supplier_and_warehouse': 6876.62 KiB (Current), 8791.39 KiB (Peak)
Transportation cost: None
